<a href="https://colab.research.google.com/github/pkrosoff/wine_machine_learning/blob/kelly_branch/wine_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,691 kB]

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("wine_hashing").getOrCreate()

In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyspark.sql.functions import regexp_replace, col


In [4]:
# from pyspark import SparkFiles
# path ="/content/drive/MyDrive/winemag-data_first150k.csv"
# spark.sparkContext.addFile(path)
# df = spark.read.csv(SparkFiles.get("winemag-data_first150k.csv"), sep=",", header=True)

path = "/content/drive/MyDrive/winemag-data_first150k.csv"
df = pd.read_csv(path)

In [5]:
df


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude
...,...,...,...,...,...,...,...,...,...,...,...
150925,150925,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio
150926,150926,France,"Offers an intriguing nose with ginger, lime an...",Cuvée Prestige,91,27.0,Champagne,Champagne,NaN,Champagne Blend,H.Germain
150927,150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Terredora
150928,150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset


In [6]:
# Examine the schema 
print("# of NaN in every column\n", df.isnull().sum(axis = 0))

# of NaN in every column
 Unnamed: 0         0
country            5
description        0
designation    45735
points             0
price          13695
province           5
region_1       25060
region_2       89977
variety            0
winery             0
dtype: int64


In [7]:
# select only id, country, price, and NOT point
df_2 = df[['description', 'price']]


In [8]:
df_2
df_2['price'].unique()
# df_2['points'].unique()

array([ 235.,  110.,   90.,   65.,   66.,   73.,   60.,   80.,   48.,
        185.,  325.,  290.,   75.,   24.,   79.,  220.,   45.,   57.,
         62.,  105.,   15.,   37.,   nan,   22.,   42.,  135.,   29.,
         23.,   17.,   26.,   55.,   39.,   69.,   30.,   50.,   40.,
        100.,   68.,   28.,   18.,   25.,   36.,   38.,   85.,   19.,
         54.,   59.,   10.,   12.,   13.,   14.,   20.,  125.,    7.,
         49.,   93.,   32.,   16.,   21.,   44.,   35.,   61.,   34.,
         11.,   58.,  120.,   41.,  200.,   43.,    9.,   56.,   46.,
         92.,   94.,   27.,   95.,   33.,   70.,  155.,   63.,  130.,
        115.,   31.,   98.,   52.,   64.,   91.,  111.,   88.,   74.,
         77.,   87.,  113.,  140.,  500.,   51.,  150.,  240.,   72.,
        103.,    8.,  136.,  141.,  102.,   47.,   53.,   96.,   78.,
        149.,  138.,  117.,    4.,   99.,  848.,  450.,  330.,  175.,
         89.,   82.,   71.,  137.,  145.,  698.,  160.,  770.,  118.,
        238.,  202.,

In [9]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150930 entries, 0 to 150929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   description  150930 non-null  object 
 1   price        137235 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.3+ MB


In [10]:
print("# of NaN in every column\n", df_2.isnull().sum(axis = 0))

# of NaN in every column
 description        0
price          13695
dtype: int64


In [11]:
# drop NaN
df_2 = df_2.dropna()
df_2

,description,price
0,This tremendous 100% varietal wine hails from ...,235.0
1,"Ripe aromas of fig, blackberry and cassis are ...",110.0
2,Mac Watson honors the memory of a wine once ma...,90.0
3,"This spent 20 months in 30% new French oak, an...",65.0
4,"This is the top wine from La Bégude, named aft...",66.0
...,...,...
150925,Many people feel Fiano represents southern Ita...,20.0
150926,"Offers an intriguing nose with ginger, lime an...",27.0
150927,This classic example comes from a cru vineyard...,20.0
150928,"A perfect salmon shade, with scents of peaches...",52.0


In [12]:
print("# of NaN in every column\n", df_2.isnull().sum(axis = 0))

# of NaN in every column
 description    0
price          0
dtype: int64


In [13]:
df_3 = df_2.assign(description_length = df_2['description'].apply(len))
df_3.info()
df_3.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137235 entries, 0 to 150929
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   description         137235 non-null  object 
 1   price               137235 non-null  float64
 2   description_length  137235 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 4.2+ MB


,description,price,description_length
0,This tremendous 100% varietal wine hails from ...,235.0,355
1,"Ripe aromas of fig, blackberry and cassis are ...",110.0,318
2,Mac Watson honors the memory of a wine once ma...,90.0,280
3,"This spent 20 months in 30% new French oak, an...",65.0,386
4,"This is the top wine from La Bégude, named aft...",66.0,376


In [14]:
# describe the price
df_3['price'].describe()

# nunuique for price
df_3['price'].nunique()

357

In [15]:
# sort price descending

price_desc = df_3.sort_values(by='price', ascending=False) 
price_desc['price'].unique()

array([2300., 2013., 1900., 1400., 1300., 1200., 1100., 1000.,  980.,
        900.,  850.,  848.,  800.,  775.,  770.,  764.,  757.,  740.,
        698.,  685.,  670.,  660.,  639.,  625.,  612.,  600.,  599.,
        596.,  588.,  580.,  574.,  550.,  545.,  544.,  535.,  530.,
        525.,  520.,  510.,  505.,  500.,  495.,  486.,  475.,  468.,
        467.,  463.,  460.,  455.,  451.,  450.,  448.,  445.,  440.,
        430.,  428.,  426.,  420.,  419.,  415.,  412.,  411.,  410.,
        400.,  399.,  398.,  395.,  391.,  390.,  385.,  380.,  376.,
        375.,  365.,  360.,  357.,  354.,  351.,  350.,  349.,  345.,
        335.,  330.,  328.,  325.,  323.,  320.,  319.,  318.,  317.,
        316.,  315.,  312.,  310.,  307.,  303.,  300.,  299.,  295.,
        294.,  293.,  292.,  290.,  289.,  288.,  285.,  283.,  282.,
        281.,  280.,  279.,  275.,  273.,  271.,  270.,  269.,  268.,
        266.,  265.,  261.,  260.,  259.,  258.,  257.,  256.,  255.,
        253.,  252.,

In [23]:
# Bin the prices so the range of prices is more manageable for the model

def price_bins(price):
    if price < 0:
        return 1
    elif price >= 35 and price < 70:
        return 2 
    elif price >= 70 and price < 105:
        return 3 
    elif price >= 105 and price < 140:
        return 5 
    elif price >= 140 and price < 175:
        return 6
    elif price >= 175 and price < 210:
        return 7
    elif price >= 210 and price < 245:
        return 8
    elif price >= 245 and price < 280:
        return 9
    elif price >= 280 and price < 315:
        return 10
    elif price >= 315 and price < 350:
        return 11
    elif price >= 350 and price < 385:
        return 12
    elif price >= 385 and price < 420:
        return 13
    elif price >= 420 and price < 455:
        return 14
    elif price >= 455 and price < 490:
        return 15
    elif price >= 490 and price < 525:
        return 16
    elif price >= 525 and price < 560:
        return 17
    elif price >= 560 and price < 595:
        return 18
    elif price >= 595 and price < 630:
        return 19
    elif price >= 630 and price < 665:
        return 20
    elif price >= 665 and price < 700:
        return 21
    else:
        return 22

#Applying transform method and assigning result to new column "points_simplified"
df_3 = df_3.assign(price_grouped = df_3['price'].apply(price_bins))
df_3.head()



,description,price,description_length,price_grouped
0,This tremendous 100% varietal wine hails from ...,235.0,355,8
1,"Ripe aromas of fig, blackberry and cassis are ...",110.0,318,5
2,Mac Watson honors the memory of a wine once ma...,90.0,280,3
3,"This spent 20 months in 30% new French oak, an...",65.0,386,2
4,"This is the top wine from La Bégude, named aft...",66.0,376,2


In [24]:
# Bring Pandas DF TO the PYSPARK

# Pandas to Spark
wine_DF = spark.createDataFrame(df_3)


wine_DF.show()



+--------------------+-----+------------------+-------------+
|         description|price|description_length|price_grouped|
+--------------------+-----+------------------+-------------+
|This tremendous 1...|235.0|               355|            8|
|Ripe aromas of fi...|110.0|               318|            5|
|Mac Watson honors...| 90.0|               280|            3|
|This spent 20 mon...| 65.0|               386|            2|
|This is the top w...| 66.0|               376|            2|
|Deep, dense and p...| 73.0|               315|            3|
|Slightly gritty b...| 65.0|               319|            2|
|Lush cedary black...|110.0|               357|            5|
|This re-named vin...| 65.0|               298|            2|
|The producer sour...| 60.0|               307|            2|
|Elegance, complex...| 80.0|               374|            3|
|From 18-year-old ...| 48.0|               260|            2|
|A standout even i...| 48.0|               289|            2|
|This wi

In [25]:
# find the number of missing values per column
from pyspark.sql.functions import isnan, when, count, col, isnull

wine_DF.select([count(when(isnull(j), j)).alias(j) for j in wine_DF.columns]).show()


+-----------+-----+------------------+-------------+
|description|price|description_length|price_grouped|
+-----------+-----+------------------+-------------+
|          0|    0|                 0|            0|
+-----------+-----+------------------+-------------+



Build the model 


In [26]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
wine_prices = StringIndexer(inputCol='price_grouped',outputCol='label')
tokenizer = Tokenizer(inputCol="description", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')



In [27]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'description_length'], outputCol='features')

In [28]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[wine_prices, tokenizer, stopremove, hashingTF, idf, clean_up])

In [29]:
# fit n transform pipeline
cleaner = data_prep_pipeline.fit(wine_DF)
cleaned = cleaner.transform(wine_DF)




In [30]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  6.0|(262145,[2701,160...|
|  3.0|(262145,[11481,33...|
|  2.0|(262145,[10077,26...|
|  1.0|(262145,[1546,423...|
|  1.0|(262145,[1546,171...|
|  2.0|(262145,[8408,181...|
|  1.0|(262145,[5561,196...|
|  3.0|(262145,[4235,315...|
|  1.0|(262145,[2306,316...|
|  1.0|(262145,[3354,415...|
|  2.0|(262145,[3848,939...|
|  1.0|(262145,[3456,215...|
|  1.0|(262145,[4176,894...|
|  2.0|(262145,[8297,120...|
|  5.0|(262145,[5847,160...|
|  2.0|(262145,[1546,439...|
| 10.0|(262145,[18911,55...|
|  2.0|(262145,[2701,104...|
|  8.0|(262145,[1546,544...|
|  2.0|(262145,[3572,190...|
+-----+--------------------+
only showing top 20 rows



In [31]:

from pyspark.ml.classification import NaiveBayes

# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)


In [32]:

# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

+--------------------+-----+------------------+-------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|         description|price|description_length|price_grouped|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------------------+-----+------------------+-------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|). Certainly not ...| 42.0|               286|            2|  1.0|[)., certainly, n...|[)., certainly, l...|(262144,[3992,268...|(262144,[3992,268...|(262145,[3992,268...|[-1515.1373063023...|[1.94451477633080...|       1.0|
|). Very good wine...| 47.0|               344|            2|  1.0|[)., very, good, ...|[)., goo

In [33]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting the wine's price was: %f" % acc)

Accuracy of model at predicting the wine's price was: 0.742975
